## Clone repository

In [ ]:
!git clone https://github.com/lamnguyenvu98/RWF2000-pytorch-version.git

Cloning into 'RWF2000-pytorch-version'...
remote: Enumerating objects: 1605, done.
remote: Counting objects: 100% (1408/1408), done.
remote: Compressing objects: 100% (1065/1065), done.
remote: Total 1605 (delta 248), reused 1384 (delta 232), pack-reused 197
Receiving objects: 100% (1605/1605), 36.38 MiB | 14.90 MiB/s, done.
Resolving deltas: 100% (368/368), done.


## Mount training data

Dataset Structure:

```
RWF2000-Small-Build (root)
    ├── train
    |    ├── Fight
    |    |     ├── data1.npz
    |    |     ├── data2.npz
    |    |     └── ...
    |    └── NonFight
    |          ├── data1.npz
    |          ├── data2.npz
    |          └── ...
    |
    └── val
         ├── Fight
         |     ├── data1.npz
         |     ├── data2.npz
         |     └── ...
         └── NonFight
               ├── data1.npz
               ├── data2.npz
               └── ...
```

In [ ]:
!python /content/drive/MyDrive/mount_cloud.py pcloud --local /content/pcloud

In [ ]:
%%capture
!unzip /content/pcloud/Datasets/ZipDataset/RWF2000-Small-Build.zip -d /content

## Create log and checkpoint directory

In [ ]:
!mkdir -p /content/log_folder
!mkdir -p /content/log_folder/checkpoints

## Create yaml file

In [ ]:
%%writefile /content/rwf2000.yaml
TRAIN:
  BATCH_SIZE: 8
  NUM_FRAMES: 64
  ACCUMULATE_BATCH: 1
  EPOCHS: 30
  LEARNING_RATE: 1.0e-2
  MOMENTUM: 9.0e-1
  WEIGHT_DECAY: 1.0e-6
SCHEDULER:
  STEP_SIZE: 3
  GAMMA: 7.0e-1
VALIDATION:
  SAVE_LAST: True
  TOP_K: 3
SETTINGS:
  ACCELERATOR: "gpu"
  DEVICES: 1
  PRECISION: 16
  RESUME: False
NEPTUNE_LOGGER:
  API_TOKEN:
  PROJECT:
  WITH_ID:
  TAGS:
DIR:
  RESUME_CHECKPOINT: '/content/log_folder/checkpoints/last.ckpt'
  CHECKPOINT_DIR: '/content/log_folder/checkpoints/'
  DATA_DIR: '/content/RWF2000-Small-Build'
  LOG_DIR: '/content/log_folder'

Overwriting /content/rwf2000.yaml


## Install missing packages on Colab

In [ ]:
!pip install lightning --quiet
!pip install torchmetrics --quiet
!pip install neptune --quiet
!pip install dotmap --quiet
!pip install "ray[serve]" --quiet

## Change dir to repo

In [ ]:
%cd /content/RWF2000-pytorch-version

/content/RWF2000-pytorch-version


## Create pyproject.toml for Colab

This file only for colab, remove installing dependencies step since Colab already has most of them.

In [ ]:
%%writefile /content/RWF2000-pytorch-version/pyproject.toml
[build-system]
requires = ["setuptools>=61.0", "setuptools-scm"]
build-backend = "setuptools.build_meta"

[project]
name = "rwf2000-pytorch"
version = "1.0.0"
authors = [
    {name = "Vu Lam Nguyen", email = "lamnguyenvu98@gmail.com"},
]
description = "Unofficial Implementation of Flow Gated Network. This network was trained on RWF2000 dataset"
requires-python = "==3.10.*"

[tool.setuptools.packages.find]
where=["src", "rayserve"]

Overwriting /content/RWF2000-pytorch-version/pyproject.toml


## Install packages

In [ ]:
!pip install .

## Train model

In [ ]:
!python train.py --config /content/rwf2000.yaml